In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [3]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
feat_for_ensemble = [raw_predictors]
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    feat_for_ensemble.append(tmp_feat_list)
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [ ]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
for key,value in var_change.items():
    print(key)
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
print(dfTrain[predictors].shape)
print(dfPred[predictors].shape)

In [4]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5,param={'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1.5,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param  = param
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =50)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [5]:
#ensemble experiment
input_round = 0
final_predictors = []
tool_dummies = []
for predictors in feat_for_ensemble:
    input_round+=1
    print(input_round)
    if input_round==1:
        for var in predictors:
            if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
                predictors.remove(var)
            elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
                predictors.remove(var)
                tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
                tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
                predictors = predictors + tmpTrain.columns.tolist()
                dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
                dfPred = pd.concat([dfPred,tmpPred],axis=1)
                if 'TOOL' in var:
                    tool_dummies += tmpTrain.columns.tolist()
    else:
        predictors+=tool_dummies
        '''for var in raw_predictors:
            if 'TOOL' in var:
                tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
                tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
                predictors = predictors + tmpTrain.columns.tolist()
                dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
                dfPred = pd.concat([dfPred,tmpPred],axis=1)'''             
    for var in predictors:
        if var not in dfPred.columns:
            dfPred[var] = 0                      
    ### model         
    tmp_test_result,tmp_result,imp = xgb_kfold(dfTrain,dfPred,predictors)
    
    tmp_result['score']=tmp_result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
    tmp_result = tmp_result[['ID','score']]
    
    tmp_test_result = tmp_test_result.rename(columns={'score':'score%d'%input_round})
    tmp_result = tmp_result.rename(columns={'score':'score%d'%input_round})
    
    final_predictors.append('score%d'%input_round)
    
    if input_round==1:
        final_test_result = tmp_test_result.copy(deep=True)
        final_result = tmp_result.copy(deep=True)
    else:
        final_test_result = final_test_result.merge(tmp_test_result[['ID','score%d'%input_round]],'inner','ID')
        final_result = final_result.merge(tmp_result[['ID','score%d'%input_round]],'inner','ID')
        
    
    
    

1
[0]	train-rmse:2.33691	eval-rmse:2.30951
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42644	eval-rmse:1.39859
[100]	train-rmse:0.879354	eval-rmse:0.852594
[150]	train-rmse:0.55223	eval-rmse:0.53205
[200]	train-rmse:0.355302	eval-rmse:0.347479
[250]	train-rmse:0.235982	eval-rmse:0.247489
[300]	train-rmse:0.163196	eval-rmse:0.1971
[350]	train-rmse:0.119154	eval-rmse:0.176113
[400]	train-rmse:0.091966	eval-rmse:0.168182
[450]	train-rmse:0.07466	eval-rmse:0.166331
[500]	train-rmse:0.062827	eval-rmse:0.166242
Stopping. Best iteration:
[476]	train-rmse:0.068024	eval-rmse:0.166025

[0]	train-rmse:2.33258	eval-rmse:2.32728
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42385	eval-rmse:1.41739
[100]	train-rmse:0.878362	eval-rmse:0.869326
[150]	train-rmse:0.552344	eva

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42214	eval-rmse:1.42517
[100]	train-rmse:0.876074	eval-rmse:0.884078
[150]	train-rmse:0.550492	eval-rmse:0.569782
[200]	train-rmse:0.354757	eval-rmse:0.3902
[250]	train-rmse:0.236347	eval-rmse:0.293632
[300]	train-rmse:0.16398	eval-rmse:0.24589
[350]	train-rmse:0.119297	eval-rmse:0.224334
[400]	train-rmse:0.092365	eval-rmse:0.214189
[450]	train-rmse:0.074875	eval-rmse:0.210015
[500]	train-rmse:0.062478	eval-rmse:0.208422
[550]	train-rmse:0.053525	eval-rmse:0.207359
[600]	train-rmse:0.046106	eval-rmse:0.207021
[650]	train-rmse:0.039985	eval-rmse:0.206836
[700]	train-rmse:0.034977	eval-rmse:0.206543
[750]	train-rmse:0.030456	eval-rmse:0.206488
[800]	train-rmse:0.026523	eval-rmse:0.206406
Stopping. Best iteration:
[789]	train-rmse:0.027347	eval-rmse:0.206367

[0]	train-rmse:2.33162	eval-rmse:2.33141
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn

[200]	train-rmse:0.353228	eval-rmse:0.390748
[250]	train-rmse:0.23562	eval-rmse:0.285913
[300]	train-rmse:0.163957	eval-rmse:0.231048
[350]	train-rmse:0.120752	eval-rmse:0.203809
[400]	train-rmse:0.094367	eval-rmse:0.189569
[450]	train-rmse:0.077214	eval-rmse:0.182751
[500]	train-rmse:0.065658	eval-rmse:0.178646
[550]	train-rmse:0.057383	eval-rmse:0.176422
[600]	train-rmse:0.050398	eval-rmse:0.175395
[650]	train-rmse:0.045168	eval-rmse:0.174729
[700]	train-rmse:0.040283	eval-rmse:0.174232
[750]	train-rmse:0.035774	eval-rmse:0.173693
[800]	train-rmse:0.031925	eval-rmse:0.173366
[850]	train-rmse:0.028552	eval-rmse:0.173208
[900]	train-rmse:0.025576	eval-rmse:0.172913
[950]	train-rmse:0.023205	eval-rmse:0.172824
[1000]	train-rmse:0.020911	eval-rmse:0.17275
[1050]	train-rmse:0.018921	eval-rmse:0.172592
[1100]	train-rmse:0.017014	eval-rmse:0.172383
[1150]	train-rmse:0.015404	eval-rmse:0.172196
[1200]	train-rmse:0.013899	eval-rmse:0.172168
[1250]	train-rmse:0.01259	eval-rmse:0.172094
[1300]	

[250]	train-rmse:0.235331	eval-rmse:0.273958
[300]	train-rmse:0.162813	eval-rmse:0.216964
[350]	train-rmse:0.118152	eval-rmse:0.188049
[400]	train-rmse:0.090252	eval-rmse:0.174814
[450]	train-rmse:0.072357	eval-rmse:0.168563
[500]	train-rmse:0.060161	eval-rmse:0.16607
[550]	train-rmse:0.050875	eval-rmse:0.164756
[600]	train-rmse:0.043759	eval-rmse:0.164101
[650]	train-rmse:0.038208	eval-rmse:0.163625
Stopping. Best iteration:
[642]	train-rmse:0.03905	eval-rmse:0.163558

Test MSE: 0.0305445378903


In [8]:
final_test_result = final_test_result.merge(dfTrain[['ID','Y']],'inner','ID')

In [9]:
test_result,result,imp = xgb_kfold(final_test_result,final_result,final_predictors,param={'max_depth':2, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1.5,'subsample':0.8,
                         'colsample_bytree':0.8})

[0]	train-rmse:2.33183	eval-rmse:2.33019
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.4232	eval-rmse:1.42109
[100]	train-rmse:0.876463	eval-rmse:0.875546
[150]	train-rmse:0.550338	eval-rmse:0.550139
[200]	train-rmse:0.36015	eval-rmse:0.36114
[250]	train-rmse:0.255009	eval-rmse:0.257523
[300]	train-rmse:0.201264	eval-rmse:0.205415
[350]	train-rmse:0.176312	eval-rmse:0.181752
[400]	train-rmse:0.164957	eval-rmse:0.171937
[450]	train-rmse:0.159646	eval-rmse:0.168274
[500]	train-rmse:0.156656	eval-rmse:0.166524
[550]	train-rmse:0.154635	eval-rmse:0.165402
[600]	train-rmse:0.153011	eval-rmse:0.165242
[650]	train-rmse:0.151719	eval-rmse:0.165041
[700]	train-rmse:0.15027	eval-rmse:0.165174
Stopping. Best iteration:
[675]	train-rmse:0.151129	eval-rmse:0.165005

[0]	train-rmse:2.3261	eval-rmse:2.35327
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stoppi

In [10]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)

In [ ]:
0.0304606416852